In [7]:
from citations.CitationClassifier import CitationClassifier
from citations.ClassificationCounter import ClassificationCounter
from documents.Corpus import Corpus
import json, pickle
import warnings, logging
from datetime import datetime
import random
from documents.Paper import *
from sys import getsizeof


In [6]:
markdown_file_path = './data/Markdown/'
foundation_models_path = './data/foundation_models.json'



classifier = CitationClassifier('allenai/multicite-multilabel-scibert')
corpus = Corpus(markdown_file_path, extensions = ['mmd'], cluster_info = (1,200))

with open(foundation_models_path, 'r') as f:
    foundational_models_json = json.load(f)
    keys, titles = list(zip(*[(key, data['title'].replace('\\infty', '∞')) for key, data in foundational_models_json.items()]))
    keys, titles = list(keys), list(titles)

100%|██████████| 1856/1856 [00:22<00:00, 83.15it/s] 


In [14]:
corpus.papers[0].__dir__()

['path',
 'content',
 'nonref_section',
 'ref_section',
 'title',
 'all_sentences',
 'references',
 'name_and_affiliation',
 '__module__',
 '__init__',
 'adjustFileContent',
 'getPaperTitle',
 'getPreAbstract',
 'normalizeNumericalCitations',
 'splitByReferenceSection',
 'getReferenceFromTitle',
 'getAllReferences',
 'getAllTextualReferences',
 'findNamesAndAffiliations',
 '__dict__',
 '__weakref__',
 '__doc__',
 '__new__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce_ex__',
 '__reduce__',
 '__getstate__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

: 

In [3]:
refresh = True

try:
    assert(not refresh)
    with open('pickle/corpus.pkl', 'rb') as f:
        corpus = pickle.load(f)
except:
    classifier = CitationClassifier('allenai/multicite-multilabel-scibert')
    corpus = Corpus(markdown_file_path, extensions = ['mmd'], limit = None)

    corpus.findAllPaperRefsAllTitles(titles = titles, keys = keys, classifier = None, results_path = results_file)

    with open('pickle/corpus.pkl', 'wb') as f:
        pickle.dump(corpus, f)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
  6%|▌         | 59/953 [00:15<04:01,  3.70it/s]


KeyboardInterrupt: 

In [ ]:
all_references = [ref for ref in corpus.getAllReferences() if ref.reference_exists and ref.title.lower() != 'ase']

resolved_citation = [ref for ref in all_references if not ref.missing_citation]
unresolved_citation = [ref for ref in all_references if ref.missing_citation]

missing_textrefs = [ref for ref in resolved_citation if len(ref.textualReferences) == 0]
missing_textrefs_unknown = [ref for ref in resolved_citation if len(ref.textualReferences) == 0 and not ref.missing_page_fail]
missing_pages = [ref for ref in missing_textrefs if ref.missing_page_fail]

perc_unresolved_citation = len(unresolved_citation)/len(all_references)
percent_missing_textref = len(missing_textrefs)/len(resolved_citation)
percent_missing_page = len(missing_pages)/len(missing_textrefs)
percent_missing_ref_unknown_cause = len(missing_textrefs_unknown)/len(resolved_citation)

print(f"Found title but cannot resolve citation for {round(perc_unresolved_citation * 100, 2)}% of cases.")
print(f"Found citation but found no uses in sentences for {round(percent_missing_textref * 100, 2)}% of cases.")
print(f"Amongst references without textual references, {round(percent_missing_page * 100,2) }% have a missing page.")
print(f"Thus in {round(percent_missing_ref_unknown_cause * 100, 2)}% of cases where we find the title and the citation, we cannot find the reference and no pages are missing.")


Found title but cannot resolve citation for 6.66% of cases.
Found citation but found no uses in sentences for 22.54% of cases.
Amongst references without textual references, 54.6% have a missing page.
Thus in 10.23% of cases where we find the title and the citation, we cannot find the reference and no pages are missing.


In [ ]:
not_missing_page = [ref for ref in missing_textrefs if not ref.missing_page_fail]
idx = random.randint(0, len(not_missing_page) - 1)
print(idx)
print(not_missing_page[idx].paper_path) 
print(not_missing_page[idx].title)
print(not_missing_page[idx].textualReferences)
print(not_missing_page[idx])
print(title_from_path(not_missing_page[idx].paper_path))

86
./Markdown/5d2dbbcccf3a47a73f746296408b500f399dc8d0.mmd
llama 2: open foundation and fine-tuned chat models
[]
Title: llama 2: open foundation and fine-tuned chat models, Citation: '['touvron et al (2022)', 'touvron et al, 2022']', missing_citation: False
5d2dbbcccf3a47a73f746296408b500f399dc8d0
Emergent inabilities? Inverse scaling over the course of pretraining


In [ ]:
corpus.bad_papers

[('./Markdown/78405722e760167df313caad7581eb70b691c495.mmd',
  AssertionError('Length of reference section matches object is 0, should be 1.')),
 ('./Markdown/9b45af10429681249fafb07c3b6012ea4ce63ffe.mmd',
  AssertionError('Length of reference section matches object is 0, should be 1.')),
 ('./Markdown/8738be33dcae524a10ccbe29afa84a870c5f6cec.mmd',
  AssertionError('Length of reference section matches object is 0, should be 1.')),
 ('./Markdown/3487c12512fa41d3a4d64f00cb842525a8590ad3.mmd',
  AssertionError('Length of reference section matches object is 0, should be 1.')),
 ('./Markdown/476f3b4816de7ce4536146b8de0d32ca6408db6f.mmd',
  AssertionError('Length of reference section matches object is 0, should be 1.')),
 ('./Markdown/cba18e04e8474b21838b076626486a6b02a68406.mmd',
  AssertionError('Length of reference section matches object is 0, should be 1.')),
 ('./Markdown/e47e63781c0e7a2c0504b9381b76b5d01b62c53d.mmd',
  AssertionError('Length of reference section matches object is 0, sh

In [ ]:
from Paper import *
paper = Paper('./Markdown/772724892819d7e6f15ce536753fdc32d022c0e0.mmd')
print(paper.nonref_section)

# a survey on image-text multimodal models

ruifeng guo

jingxuan wei

jingxuan wei

linzhuang sun

bihui yu

guiyong chang

dawei liu

sibo zhang

zhengbing yao

mingjun xu

liping bu

###### abstract

amidst the evolving landscape of artificial intelligence, the convergence of visual and textual information has surfaced as a crucial frontier, leading to the advent of image-text multimodal models. this paper provides a comprehensive review of the evolution and current state of image-text multimodal models, exploring their application value, challenges, and potential research trajectories. initially, we revisit the basic concepts and developmental milestones of these models, introducing a novel classification that segments their evolution into three distinct phases, based on their time of introduction and subsequent impact on the discipline. furthermore, based on the tasks' significance and prevalence in the academic landscape, we propose a categorization of the tasks associated with i

In [ ]:
def in_list(value, L):
    for el in L:
        if value in el:
            return True
    return False